## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [4]:
# Create a list of at least 10 column names to use as X data
selected_columns = [
    'Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
    'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'NumCompaniesWorked',
    'PercentSalaryHike', 'TotalWorkingYears', 'WorkLifeBalance'
]

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes


,0
Age,int64
DistanceFromHome,int64
Education,int64
EnvironmentSatisfaction,int64
JobInvolvement,int64
JobLevel,int64
JobSatisfaction,int64
NumCompaniesWorked,int64
PercentSalaryHike,int64
TotalWorkingYears,int64


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_df = X_df.apply(pd.to_numeric, errors='coerce')

In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)


In [10]:
# Create a OneHotEncoder for the Department column
encoder_department = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder_department.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = encoder_department.transform(y_train[['Department']])
y_test_department = encoder_department.transform(y_test[['Department']])



In [12]:
# Create a OneHotEncoder for the Attrition column
encoder_attrition = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder_attrition.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = encoder_attrition.transform(y_train[['Attrition']])
y_test_attrition = encoder_attrition.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [13]:
# Find the number of columns in the X training data
num_features = X_train.shape[1]

# Create the input layer
inputs = layers.Input(shape=(num_features,))

# Create at least two shared layers
shared_layer_1 = layers.Dense(32, activation='relu')(inputs)
shared_layer_2 = layers.Dense(16, activation='relu')(shared_layer_1)

In [14]:
# Create a branch for Department # with a hidden layer and an output layer
# Create the hidden layer
department_branch = layers.Dense(8, activation='relu')(shared_layer_2)

# Create the output layer
department_output = layers.Dense(y_train_department.shape[1], activation='softmax', name='department_output')(department_branch)



In [15]:
# Create a branch for Attrition # with a hidden layer and an output layer
# Create the hidden layer
attrition_branch = layers.Dense(8, activation='relu')(shared_layer_2)

# Create the output layer
attrition_output = layers.Dense(y_train_attrition.shape[1], activation='softmax', name='attrition_output')(attrition_branch)


In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

In [17]:
# Create the model
model = Model(inputs=inputs, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam', loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'}, metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 11)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 32)             │            384 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 16)             │            528 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 8)              │            136 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 8)              │            136 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             27 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             18 │ dense_3[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,229 (4.80 KB)

 Trainable params: 1,229 (4.80 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Train the model
model.fit(X_train, [y_train_department, y_train_attrition], epochs=100, batch_size=10)
# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_df, y_df),
#     epochs=epochs
# )

Epoch 1/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - attrition_output_accuracy: 0.8757 - department_output_accuracy: 0.7078 - loss: 0.9525
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - attrition_output_accuracy: 0.8859 - department_output_accuracy: 0.7189 - loss: 0.9524
Epoch 3/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - attrition_output_accuracy: 0.8803 - department_output_accuracy: 0.7119 - loss: 0.9521
Epoch 4/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8675 - department_output_accuracy: 0.7152 - loss: 0.9621
Epoch 5/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - attrition_output_accuracy: 0.8771 - department_output_accuracy: 0.7100 - loss: 0.9198
Epoch 6/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - attrition_output_accuracy: 0.8752 - department_output_accuracy: 0.6956 - loss: 0.9610
Epoch 7/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - attrition_output_accuracy: 0.8870 - department_output_accuracy: 0.7409 - loss: 0.9193
Epoch 8/100
1

In [22]:
# Evaluate the model with the testing data
model_loss, department_accuracy, attrition_accuracy = model.evaluate(X_test, [y_test_department, y_test_attrition])


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - attrition_output_accuracy: 0.8406 - department_output_accuracy: 0.6683 - loss: 1.5572


In [23]:
# Print the accuracy for both department and attrition
print(f'Department Accuracy: {department_accuracy}')
print(f'Attrition Accuracy: {attrition_accuracy}')

Department Accuracy: 0.8503401279449463
Attrition Accuracy: 0.680272102355957


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?



```
# This is formatted as code
```

YOUR ANSWERS HERE

1. Accuracy might not be the best metric to use on this data, especially if there is class imbalance between the categories in the Department and Attrition columns. Alternative metrics such as precision, recall, or F1-score could provide additional insights into the model's performance.

2. I used the 'softmax' activation function for both output layers, as it is suitable for multi-class classification tasks. It allows the model to output a probability distribution over the predicted classes for both the Department and Attrition targets.

3. The model could be improved by adjusting the architecture, such as adding more hidden layers or increasing the number of neurons. Additionally, hyperparameter tuning, using different activation functions, and experimenting with different optimizers could enhance performance. Collecting more data or using techniques to handle class imbalance might also be beneficial.